**Baseline Model**

In this notebook we are going to build a reccomendation system only based on the "tags" without the including the NLP techniques on "discription"

In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.append(project_root)

from src import config
RAW_DATA_PATH = config.RAW_DATA_PATH

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
try:
    df = pd.read_csv(RAW_DATA_PATH)

except FileNotFoundError:
    print("File not found. Please check the file path and try again.")

In [4]:
df_baseline = df[['title' , 'tags']].copy()
print(f"Data shape: {df_baseline.shape}")
df_baseline.head()

Data shape: (70948, 2)


,title,tags
0,Salad Days (Tang LiuZang) - Part 2,"['BL', 'Manhua', 'Romance', 'Shounen-ai', 'Spo..."
1,The Master of Diabolism,"['Action', 'Adventure', 'BL', 'Comedy', 'Manhu..."
2,JoJo's Bizarre Adventure Part 7: Steel Ball Run,"['Action', 'Adventure', 'Horror', 'Mystery', '..."
3,A Sign of Affection,"['Romance', 'Shoujo', 'Slice of Life', 'Disabi..."
4,Moriarty the Patriot,"['Mystery', 'Shounen', 'Detectives', 'England'..."


**Basic Modelling (CountVectorizer)**

-Only look for the interested tag not the importance of the tag



**Why is is suitable for baseline**

-Fast and the computing cost is low

In [5]:
count_vectorizer = CountVectorizer(stop_words='english')
count_matrix = count_vectorizer.fit_transform(df_baseline['tags'])
print(f"Count Matrix shape: {count_matrix.shape}")

Count Matrix shape: (70948, 653)


In [6]:
sample_size = 10000
df_sample = df_baseline.iloc[:sample_size].reset_index(drop=True)
count_matrix_sample = count_matrix[:sample_size]

In [7]:
cosine_sim_baseline = cosine_similarity(count_matrix_sample, count_matrix_sample)

In [8]:
indices = pd.Series(df_sample.index, index=df_sample['title']).drop_duplicates()

In [9]:
def get_baseline_rec(title, cosine_sim = cosine_sim_baseline):
    if title not in indices:
        return "Title not found in the dataset."
    idx = indices[title]

    if isinstance(idx, pd.Series):
        idx = idx.iloc[0]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:6]

    rec_indices = [i[0] for i in sim_scores]

    return df_sample['title'].iloc[rec_indices]

**Baseline Outcomes**

In [10]:
test_title = df_sample['title'].iloc[10]

In [11]:
print(f"Chosen Title: {test_title}\n")
print("Top 5 Recommendations:")
print(get_baseline_rec(test_title))

Chosen Title: Tower of God - Part 3

Top 5 Recommendations:
1129               ASURA (Handol)
2841    Crown Prince of Myolyeong
5134                         Flow
5305             The Star Seekers
827             Magician - Part 2
Name: title, dtype: object


**Outcome Evaluation**

Baseline model is only interested in the tags so the reccomendations may be irrevelent to the chosen title's content

**Shortcomings**

-Description is missing

-Ratings are not included

-Sentiments are not included